# pyhelios Demo: Live LiDAR Simulation Trajectory Plot

Notebook: Mark Searle, 2021

***

This notebook demos the excecution of a __LiDAR survey simulation__ using integrated Python functionalities to initiate __HELIOS++__.
The trajectory of the virtual scanner which can be configured to sit upon different moving or stationary platforms is plotted __live__ using __matplotlib__.

***
The survey is configured through a __.xml survey file__ containing survey name, platform type, scanner type, and scene to be scanned along with platform and scanner settings.  
The simulation runs in several legs which represent individual scanner trajectories and are also configured from within survey file.

__The survey files are located within 'helios/data/surveys'!__

The __survey used in this demo__ (`custom_als_toyblocks.xml`) should be located in this survey folder.

***

In [ ]:
import time
import numpy as np
import pyhelios

In [ ]:
import os
os.chdir("..")

In [ ]:
# Survey to be used.
survey_path = 'toyblocks/custom_als_toyblocks.xml'

## Live Trajectory Plot of LiDAR Simulation with PyHelios

***

1. Define Callback function to continuously extract values from simulation:

In [ ]:
# Empty list for trajectory values.
tpoints = []

In [ ]:
# Callback frequency is defined through parameter simFrequency which is set once the simulation is built.
def callback(output=None):
    global tpoints
    
    # Store trajectories in variable.
    trajectories = output[1]
    # Add current values to list
    try: 
        tpoints.append([trajectories[len(trajectories) - 1].position[0],
                        trajectories[len(trajectories) - 1].position[1],
                        trajectories[len(trajectories) - 1].position[2]])
        
    # At the beginning of each new simulation leg, trajectories can have the length 0.
    # When indexing with [0 - 1] an indexing issue can occurr that is caught in this block.
    except Exception as err:
        #print('An Error occurred:', err)
        pass

1. Configure simulation context and build a simulation:

In [ ]:
# Sim context.
# Set logging.
# pyhelios.loggingVerbose()
# Set seed for random number generator.
pyhelios.default_rand_generator_seed("123")

In [ ]:
# Build simulation parameters: (surveyPath, assetsPath, outputPath, ...).
from pyhelios import SimulationBuilder

simB = SimulationBuilder('data/surveys/' + survey_path, "assets/", "output/")
simB.setLasOutput(True)
simB.setZipOutput(False)
simB.setCallbackFrequency(10000)
simB.setFinalOutput(True)
simB.setCallback(callback)

2. Load survey file:

In [ ]:
sim = simB.build()

3. Create ground plane object for more comprehensive visualization:

In [ ]:
# Empty list for groundplane vals.
groundplane_list = []

# List filled every 25th value from (-75, -75, 0) to (75, 75, 0).
for i in range(-75, 76, 25):
    for j in range(-75, 76, 25):
        groundplane_list.append([i, j, 0])
        
# List converted to numpy Array.
groundplane = np.array(groundplane_list)

4. Define function __update_3dline__ that adds values to a line object and plots them onto a matplotlib figure from a continuously updating list.

In [ ]:
def update_3dline(dynamic_list, line_obj, sim, hdisplay, refresh_rate=0.5):
    """Continuously updates matplotlib line with values from a steadily growing list.
     The function terminates when no new values are added to the list within the specified refresh rate.
    
    Parameters:
    dynamic_list (list): A list of xyz points that is continuously growing in the background (format: [[x, y, z], [x, y, z], ...])
    line_obj (matplotlib.axes.Axes.plot): An empty line with 3 dimensions. E.g.: l, = ax.plot([], [], [])
    sim (pyhelios.simulation_build.SimulationBuild): The pyhelios simulation object to determine when to stop the live plot
    hdisplay (IPython.core.display_functions.DisplayHandle): The display handle to update
    
    Optional:
    refresh_rate (float, default=0.5): Determines time interval in seconds between each iteration of function.
    """
    while sim.isRunning():
        time.sleep(float(refresh_rate))
                    
        # Convert x, y, z list to array.
        a = np.array(dynamic_list)
        
        # Update values of line with data from array.
        line1.set_xdata(a[:,0])
        line1.set_ydata(a[:,1])
        line1.set_3d_properties(zs=a[:,2])
        
        # Update figure
        hdisplay.update(fig)

4. Initiate matplotlib figure and run simulation. Function from previous cell used to update line.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython import display

# Matplotlib figure.
fig = plt.figure()

# Axes3d axis onto mpl figure.
ax = fig.add_subplot(111, projection='3d')

# Set axis extent.
ax.set_xlim([-75,75])
ax.set_ylim([-75,75])
ax.set_zlim([0, 100])

# Add axis labels.
ax.set_xlabel('$X$')
ax.set_ylabel('$Y$')
ax.set_zlabel('$Z$')

# Set title.
ax.text2D(0.16, 1, "LIVE:", fontsize='11', transform=ax.transAxes, c='r')
ax.text2D(0.27, 1, 'pyhelios LiDAR Simulation Trajectory Plot', fontsize='11', transform=ax.transAxes)

# Plot groundplane onto figure.
ax.plot_trisurf(groundplane[:,0], groundplane[:,1], groundplane[:,2], color='darkgoldenrod', label='groundplane', alpha=0.5)

# Start the simulation.
sim.start()

time.sleep(0.2)
hdisplay = display.display("", display_id=True)

# Create empty line object.
line1, = ax.plot([], [], [])

# Run function to plot trajectory while simulation is running.
update_3dline(tpoints, line1, sim, hdisplay=hdisplay, refresh_rate=0.1)

plt.close(fig)